In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".50"
# jax.config.update("jax_default_device", jax.devices()[1])

import numpy as np
import matplotlib.pyplot as plt
import yaml
# import meshio
from pathlib import Path
from tqdm.auto import tqdm

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from jax import grad, vmap, random
import optax
from tqdm.auto import tqdm
from typing import Callable
import pickle
import equinox as eqx

from triangular_transport.flows.flow_trainer import (
    NNTrainer,
)

from triangular_transport.flows.interpolants import (
    linear_interpolant,
    linear_interpolant_der,
    trig_interpolant,
    trig_interpolant_der,
    sigmoid_interpolant,
    sigmoid_interpolant_der,
)
from triangular_transport.flows.loss_functions import vec_field_loss
# from triangular_transport.networks.flow_networks import MLP, MLP_Triangular
from triangular_transport.flows.methods.utils import UnitGaussianNormalizer
from triangular_transport.flows.dataloaders import gaussian_reference_sampler
from triangular_transport.kernels.kernel_tools import get_gaussianRBF, vectorize_kfunc

import json
import h5py

import argparse

plt.style.use("ggplot")

In [11]:
def read_data_h5(path="data.h5"):
    with h5py.File(path, "r") as f:
        targets = f["/target"][...]
        data = f["/data"][...]
    return targets, data

with open("poisson.yaml") as fid:
    inargs = yaml.full_load(fid)

utrue = np.load("training_dataset/true_param_grid.npy")
ytrue = np.load("training_dataset/true_state_grid.npy")
map_est = np.load("training_dataset/map_param_grid.npy")
targets, yobs = read_data_h5()

nsamples = 20000

hmala_samps = np.load("training_dataset/chain_30/hmala_samples_grid_delta.npy").reshape(nsamples, 33, 33)
hmala_mean = np.mean(hmala_samps, axis=0)
hmala_var = np.var(hmala_samps, axis=0)

train_dim = 50000
nx = ny = 33
flat_length = nx * ny
ys = jnp.expand_dims((np.load("training_dataset/solutions_grid_delta.npy"))[:train_dim], 1)
us = (np.load("training_dataset/parameters_delta.npy"))[:train_dim, :]

us_ref = us.copy()
np.random.shuffle(us_ref)
us = jnp.expand_dims(us, 1)
us_ref = jnp.expand_dims(us_ref, 1)

us_test = (np.load("training_dataset/parameters_delta.npy"))[train_dim:train_dim + 20000, :]
np.random.shuffle(us_test)
us_test = jnp.expand_dims(us_test, 1)

ys_normalizer = UnitGaussianNormalizer(ys)
ys_normalized = ys_normalizer.encode()

In [13]:
from unet import UNet
key = random.PRNGKey(seed=42)
key1, key2 = random.split(key=key, num=2)
batch_size = 128
steps = 50000
print_every = 5000
yu_dimension = (100, nx * ny)
dim = yu_dimension[0] + yu_dimension[1]
hidden_layer_list = [512] * 4
model = UNet(
    data_shape=(1, nx, ny),
    is_biggan=False,
    dim_mults=[1, 2, 4],
    hidden_size=64,
    heads=4,
    dim_head=32,
    dropout_rate=0.0,
    num_res_blocks=2,
    attn_resolutions=[16],
    langevin=False,
    key=key
)
schedule = optax.warmup_cosine_decay_schedule(
    init_value=0.0,
    peak_value=3e-4,
    warmup_steps=2_000,
    decay_steps=steps,
    end_value=1e-5,
)
optimizer = optax.adamw(schedule)
optimizer = optax.chain(optax.clip_by_global_norm(1.0), optax.adamw(schedule))
interpolant = linear_interpolant
interpolant_der = linear_interpolant_der
interpolant_args = {"t": None, "x1": None, "x0": None}

SyntaxError: invalid syntax (unet.py, line 476)